# Consolidate data

This notebook processes the original data into a consolidated databaset. The geometries are extracted from the initial `.gml` files and are stored into a `sqlite` database (`cadastro.db`), with an index for the municipality ID and the geometries stored as well-known-blurb strings.

In [18]:
%matplotlib inline

import os
import tools
import sqlite3
import pandas as pd
import geopandas as gpd
from sqlalchemy import create_engine

in_folder = '../../01 Inspire buildings maps/Raw maps No PVasco Navarra/'
out_folder = '/media/dani/baul/BigData/cadastro_esp/'
out_folder = '/Users/dani/Desktop/'
engine = create_engine('sqlite:////'+out_folder+'cadastro.db')

## Extract all data into single table

* Dump all data from `.gml` files

In [3]:
%%time
files = [i for i in os.listdir(in_folder) if '.zip' in i]

for f in files:
    try:
        db = tools.extract_geoms(in_folder + f, encode_geom=True)
        db['file'] = f.replace('.zip', '')
        db.to_sql('cadastro', engine, if_exists='append', 
                  index=False, chunksize=100000)
    except:
        print('File %s failed\n'%f)

File A.ES.SDGC.BU.25218.zip failed

File A.ES.SDGC.BU.39103.zip failed

CPU times: user 1h 27min 40s, sys: 2min 42s, total: 1h 30min 23s
Wall time: 6h 48min 1s


Upon inspection, `A.ES.SDGC.BU.39103.gml` is an empty file. For the case of `A.ES.SDGC.BU.25218.zip`, there is a date column with a wrong year (`-1900`), which throws an error. Since it is not possible to turn off `convert_dates`, we manually replace `-1900` by `1900` in the `.geojson` created, and write it to the database:

In [22]:
fi = open('A.ES.SDGC.BU.25218.geojson', 'r')
fo = open('A.ES.SDGC.BU.25218_fixed.geojson', 'w')
fixed = fi.read().replace('-1900', '1900')
fo.write(fixed)
fo.close()
fi.close()

db = gpd.read_file('A.ES.SDGC.BU.25218_fixed.geojson')
db = db.to_crs(epsg=25830)
xys = pd.DataFrame([[pt.x, pt.y] for pt in db.centroid],
                   columns=['X', 'Y'])
db['geometry'] = db['geometry'].apply(lambda g: g.to_wkb())
db = pd.DataFrame(db)
db = db.join(xys)
db.to_sql('cadastro', engine, if_exists='append', 
          index=False, chunksize=100000)

Then we remove manually the remaining files:

In [23]:
! rm A.ES.SDGC.BU.25218*
! rm A.ES.SDGC.BU.39103.*

* Create indices

In [24]:
%%time
# http://pythoncentral.io/introductory-tutorial-python-sqlalchemy/
conn = sqlite3.connect(out_folder+'cadastro.db')
c = conn.cursor()

c.execute("CREATE INDEX munID ON cadastro (file);")
c.execute("CREATE INDEX xID ON cadastro (X);")
c.execute("CREATE INDEX yID ON cadastro (Y);")

CPU times: user 1min 11s, sys: 26.8 s, total: 1min 38s
Wall time: 4min 16s


* Write CRS information

In [25]:
db_crs = pd.DataFrame({'crs': [25830]}, index=['epsg'])
db_crs.index.name = 'format'
db_crs.to_sql('crs', engine, if_exists='replace')
db_crs

,crs
format,
epsg,25830


* Write municipality code/name information into separate table (`code2name`)

In [26]:
url = ('../../01 Inspire buildings maps/Raw maps No PVasco Navarra/'\
       '00 Municipalities with data.xml')
code2name = tools.parse_municipalities(url)\
                 .reset_index()\
                 .rename(columns={'index': 'code',
                                  'code2name': 'name'})
code2name.to_sql('code2name', engine, if_exists='replace',index=False)
code2name.head(2)

,code,name
0,02001,ABENGIBRE
1,02002,ALATOZ
